# Example of a pretrained RNN character model. 

This example show a pre-trained RNN making predictions. 

#### Data and preprocessing
The text for training has been extracted for the Phd thesis:

Joller-Graf, Klaus, Herrn Prof Dr Wilfried Schley, and Frau Prof Dr Ingeborg Kriwet. "Didaktik des integrativen Unterrichts."

Which can be downloaded from: http://edudoc.ch/record/3408/files/zu05056.pdf

Creating the text-file from the pdf via:
```
    ~/Downloads/xpdfbin-mac-3.04/bin64/pdftotext  -enc UTF-8 zu05056.pdf
```
http://stackoverflow.com/questions/4039930/how-to-save-text-file-in-utf-8-format-using-pdftotext

This text has been preprocessed with 
```
X, Y, char_idx = \
    textfile_to_semi_redundant_sequences(path, seq_maxlen=maxlen, redun_step=3)
```
yielding
```
Text total length: 1418267
Distinct chars: 109
Total sequences: 472748
```

### The network
Here we use the trained network, which we obtained using the library tflearn, as follows:

#### Definition
The network has been defined in tflearn as:
```
g = tflearn.input_data([None, maxlen, len(char_idx)])
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512)
g = tflearn.dropout(g, 0.5)
g = tflearn.fully_connected(g, len(char_idx), activation='softmax')
g = tflearn.regression(g, optimizer='ADAM', loss='categorical_crossentropy',
                       learning_rate=0.001)

m = tflearn.SequenceGenerator(g, dictionary=char_idx,
                              seq_maxlen=maxlen,
                              clip_gradients=5.0,
                              checkpoint_path='model_shakespeare')
```
Note that the network guesses the next letter using all, the hidden states of all neurons of the last layer. 

#### Training
...and fitted (50 epochs) using:
```
for i in range(50):
        seed = random_sequence_from_textfile(path, maxlen)
        m.fit(X, Y, validation_set=0.1, batch_size=128,
              n_epoch=1, run_id='shakespeare')
        print("-- TESTING...")
        print("-- Test with temperature of 1.0 --")
        print(m.generate(600, temperature=1.0, seq_seed=seed))
        print("-- Test with temperature of 0.5 --")
        print(m.generate(600, temperature=0.5, seq_seed=seed))    
```

#### Freezing
finally the model has been frozen with:
```
# Loading the pretrained model from the checkpoint
m.load('/home/dueo/Dropbox/__ZHAW/Projekte/RNN/model_shakespeare-166250')
sess = m.session
graph = tf.get_default_graph()
input_graph_def = graph.as_graph_def()
from tensorflow.python.framework import graph_util
# The output node names are used to determine which 
# part of the graph needs to be frozen.
output_node_names = "FullyConnected/Softmax"
output_graph_def = graph_util.convert_variables_to_constants(
        sess, # The session is used to retrieve the weights
        input_graph_def, # The graph_def is used to retrieve the nodes
        output_node_names.split(",") 
    )

with tf.gfile.GFile('didactic_25.pb', "wb") as f:
    f.write(output_graph_def.SerializeToString())
```

In [1]:
import tensorflow as tf
from __future__ import print_function
import os
import numpy as np

The network ..

In [2]:
maxlen = 25 #The maximal lenth of the sequence

In [3]:
import sys
try:
    import cPickle as pickle
except ImportError:
    import pickle
import gzip

with open('zu05056_char_idx.pkl', 'rb') as f:
    if sys.version_info.major > 2:
        char_idx = pickle.load(f, encoding='latin1')
    else:
        char_idx = pickle.load(f)
        
list(char_idx.keys())[20:30],list(char_idx.values())[20:30]

(['T', 'F', 'e', 'I', ',', 'a', 'r', ' ', '[', '©'],
 [15, 67, 103, 95, 5, 101, 77, 3, 45, 59])

### Downloading of the network

In [4]:
# Downloading the model, if it does not exist
import urllib
import os
if not os.path.isfile('didactic_25.pb'):
    urllib.urlretrieve("https://dl.dropboxusercontent.com/u/9154523/models/rnn_fun/didactic_25.pb", "didactic_25.pb")
%ls -hl didactic_25.pb

-rw-r--r-- 1 root root 22M Jan  9 10:01 didactic_25.pb


In [5]:
with tf.gfile.GFile('didactic_25.pb', "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    
tf.reset_default_graph()
graph = tf.Graph().as_default() 
tf.import_graph_def(graph_def,  name='')

In [6]:
#ops = tf.get_default_graph().get_operations()
#for i in ops:print(i.name)

In [7]:
graph = tf.get_default_graph()
feed = graph.get_tensor_by_name('InputData/X:0')
fetch = graph.get_tensor_by_name('FullyConnected/Softmax:0')

In [8]:
# The seed for prediction. Note that the seed need to be exactely of length maxlen
seed = 'Die Grundlagen war dabei '[0:maxlen]
seed, len(seed)

('Die Grundlagen war dabei ', 25)

In [9]:
# Creating a one-hot-encoded matrix
X = np.zeros((1, maxlen, len(char_idx))) #One Batch, t, X_t (one-got-encoded)
for t, char in enumerate(seed):
    X[0, t, char_idx[char]] = 1.0  

In [10]:
with tf.Session() as sess:
    pred = sess.run(fetch, feed_dict={feed:X})

In [11]:
nl = np.argmax(pred) #next letter
nl

18

In [12]:
# Code taken from from tflearn
def reverse_dictionary(char_idx):
    # Build reverse dict
    rev_dic = {}
    for key in char_idx:
        rev_dic[char_idx[key]] = key
    return rev_dic

rev_dic = reverse_dictionary(char_idx)
rev_dic[nl]

'd'

In [13]:
def _sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    logit = np.log(a) 
    p = np.exp(logit / temperature)
    #1.001 to be on the save side, sum(p) < 1 for np.random.multinomial
    p /= (1.001 * np.sum(p))
    return np.argmax(np.random.multinomial(1, p, 1))

n = _sample(pred[0])
n, rev_dic[n]

(18, 'd')

In [14]:
import sys
# Code adapted from tflearn SequenceGenerator
def generate(sess, seq_seed, show=True, seq_length = 400, temperature = 0.1,  seq_maxlen=25):
    sequence = seq_seed
    generated = seq_seed
    dic = char_idx
    rev_dic = reverse_dictionary(dic)


    whole_sequence = seq_seed

    for i in range(seq_length):
        X = np.zeros((1, seq_maxlen, len(dic)))
        for t, char in enumerate(sequence):
            X[0, t, dic[char]] = 1.
        preds = sess.run(fetch, feed_dict={feed:X})[0] #Getting next letter distribution
        next_index = _sample(preds, temperature) #Sampling a letter from the distribution
        #next_index = np.argmax(preds)
        next_char = rev_dic[next_index]
        if show:
            sys.stdout.write(next_char)
            sys.stdout.flush()
        generated += next_char
        sequence = sequence[1:] + next_char
        whole_sequence += next_char
    return whole_sequence

with tf.Session() as sess:
    res = generate(sess, seed, temperature=1.0)
    print('\n')
    print(res)

das Er

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


fahrungsigen: In: A. Eberwein, SchÃ¼ler (Handollal, E. & Schley, B. & Heuten 1997) arbeitet â auf Wahrnehmung in der Sachkompetenz der SchÃ¼ler (hier um das Verhalten gehalten wurde. Die MÃ¶glichkeit finden ypielen Troten gehen und ausreichend die Gruppenvermittlung und Lehrerin dyraus wieder als Arbeiten. Diese Verleding mit Schulkompetenz - eine Methoden auf die Fertigkeiten, die einer b

Die Grundlagen war dabei das Erfahrungsigen: In: A. Eberwein, SchÃ¼ler (Handollal, E. & Schley, B. & Heuten 1997) arbeitet â auf Wahrnehmung in der Sachkompetenz der SchÃ¼ler (hier um das Verhalten gehalten wurde. Die MÃ¶glichkeit finden ypielen Troten gehen und ausreichend die Gruppenvermittlung und Lehrerin dyraus wieder als Arbeiten. Diese Verleding mit Schulkompetenz - eine Methoden auf die Fertigkeiten, die einer b


### Some further examples

In [15]:
ts = (1.0, 1.0, 0.5, 0.5, 0.1, 0.05)
with tf.Session() as sess:
    for t in ts:
        print()
        print("Temperature {}".format(t))
        print(generate(sess, seed, temperature=t, show=False))


Temperature 1.0


/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


Die Grundlagen war dabei diskutiert. Da nicht mit dem Achten zusammeny Anspruch durch persÃ¶nliche Arbeit entgegenser Pinge und insgesamt nehmen Didaktik stÃ¼rzen. Abgeschlossen, dass sie nicht sehr stark aufgebauten, die bise, dass Bereitschaftsbereichen Texter zu kommen ist eine zu kritische Prinzipien zu erkennen und in dieser Grundlage, gibt. NatÃ¼rlich kann in diesem Anspruch, das Probleme vor, die Portfellen im etwa

Temperature 1.0
Die Grundlagen war dabei dem Phase entscheiden. Ich dem Wohlung des zusammengesetzt die Ergebnisse nach dem Menschen ihrer Entscheide gegrafiert werden, die sie zu eigenen Fragen gebenâ (Bianca L./MS1).
âUnd mit weisenden heisst, erste LernmÃ¶glichkeiten erkennen. In seiner Kleinigen ist ein frei zu werden fest, dass die Kinder wird die integrierten SchÃ¼lerinnen und SchÃ¼ler. âJe nach dem Lehren und Fertigkeiten nÃ

Temperature 0.5
Die Grundlagen war dabei die SchÃ¼lerinnen und SchÃ¼ler auf den Punkt in der Schule erwÃ¤hnt wird. Die eine Klasse

#### Some observations:

* For low temperatures, the system gets trapped in a kind of local minima.
* At least it's political correct:'Schülerinnen und Schüler, 'Heilpädagogin oder dem Heilpädagogen'